In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tsunamibayes as tb

In [2]:
bounds = {'lat_min':-10,'lat_max':-2,'lon_min':126,'lon_max':133.5}
# depth_path = 'sum_slab2_dep_02.23.18.xyz'
# dip_path = 'sum_slab2_dip_02.23.18.xyz'
# strike_path = 'sum_slab2_str_02.23.18.xyz'
# # arrays = tb.fault.load_slab2_data(depth_path,dip_path,strike_path,bounds)
arrays = np.load('../scenarios/banda_1852/data/slab2_banda.npz')
banda = tb.GridFault(bounds=bounds,**arrays)

In [3]:
# np.savez('slab2_banda.npz',**arrays)

In [4]:
Lat,Lon = np.meshgrid(banda.lat,banda.lon,indexing='ij')

In [3]:
lon = 131.46686782132107
lat = -3.994735053198803
mag = 8.828132593154196
delta_logl = -0.018864928329097086
delta_logw = -0.08955551930345311
depth_offset = 10.764981349160156

In [4]:
length = tb.utils.calc_length(mag, delta_logl)
width = tb.utils.calc_width(mag, delta_logw)
slip = tb.utils.calc_slip(mag, length, width)

In [11]:
subfault_params = banda.subfault_split2(lat,lon,length,width,slip,depth_offset=0,slip_dist='elliptical',rake='parallel',n=5)

In [16]:
subfault_params

In [12]:
fault = tb.maketopo.make_fault_dtopo(subfault_params,bounds,verbose=True)

## Uniform slip, rake = 90

In [18]:
dtopo = fault.dtopo
X,Y = np.meshgrid(dtopo.x,dtopo.y)
fault.plot_subfaults(slip_color=True,plot_rake=True)
# plt.pcolormesh(X,Y,dtopo.dZ[0])
# plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

In [20]:
# fault.plot_subfaults(slip_color=True)
plt.pcolormesh(X,Y,dtopo.dZ[0])
plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
plt.show()

## Uniform slip, parallel slip vectors (variable rake)

In [24]:
dtopo = fault.dtopo
X,Y = np.meshgrid(dtopo.x,dtopo.y)
fault.plot_subfaults(slip_color=True,plot_rake=True)
# plt.pcolormesh(X,Y,dtopo.dZ[0])
# plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

In [25]:
# fault.plot_subfaults(slip_color=True)
plt.pcolormesh(X,Y,dtopo.dZ[0])
plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

## Elliptical slip, rake=90

In [7]:
dtopo = fault.dtopo
X,Y = np.meshgrid(dtopo.x,dtopo.y)
fault.plot_subfaults(slip_color=True,plot_rake=True)
# plt.pcolormesh(X,Y,dtopo.dZ[0])
# plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

In [8]:
# fault.plot_subfaults(slip_color=True)
plt.pcolormesh(X,Y,dtopo.dZ[0])
plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

## Elliptical slip, variable rake

In [13]:
dtopo = fault.dtopo
X,Y = np.meshgrid(dtopo.x,dtopo.y)
fault.plot_subfaults(slip_color=True,plot_rake=True)
# plt.pcolormesh(X,Y,dtopo.dZ[0])
# plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

In [14]:
# fault.plot_subfaults(slip_color=True)
plt.pcolormesh(X,Y,dtopo.dZ[0])
plt.colorbar()
# cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
# plt.clabel(cs)
plt.show()

In [17]:
dtopo.dZ.shape

In [8]:
fault.plot_subfaults()

In [9]:
fault.plot_subfaults_depth()

In [13]:
fault?

In [9]:
fault.plot_subfaults?

In [17]:
def get_length(deltalogl, mag):
    """ Length is sampled from a truncated normal distribution that
    is centered at the linear regression of log10(length_cm) and magnitude.
    Linear regression was calculated from wellscoppersmith data.

    Parameters:
    mag (float): the magnitude of the earthquake

    Returns:
    length (float): Length in meters. a sample from the normal distribution centered on the regression
    """

    m = 0.5233956445903871       # slope
    c = 1.0974498706605313     # y intercept

    mu_logl = m*mag + c
    logl = mu_logl + deltalogl
    return 10**logl

def get_width(deltalogw, mag):
    """
    Parameters:
    mag (float): the magnitude of the earthquake

    Returns:
    width (float): width in meters
    """
    m = 0.29922483873212863       # slope
    c = 2.608734705074858     # y intercept

    mu_logw = m*mag + c
    logw = mu_logw + deltalogw
    return 10**logw

In [20]:
def get_slip(length, width, mag):
    """Calculated from magnitude and rupture area, Ron Harris gave us the equation
        Parameters:
        Length (float): m
        Width (float): m
        mag (float): moment magnitude

        Return:
        slip (float): meters
        """
    #Dr. Harris' rigidity constant : 3.2e11 dynes/cm^2
    mu_dyn_cm2 = 3.e11
    mu = mu_dyn_cm2 * 1e-5 * 1e4 #convert to N/m^2
    slip = 10**(3/2 * ( mag + 6.06 )) / (mu * length * width)
    return slip

In [21]:
mag = 8.786524
lon = 132.048556
lat = -4.558317
deltalogl = -0.011334
deltalogw = 0.017210
deltadepth = 1020.248

In [25]:
length = get_length(deltalogl,mag)
width = get_width(deltalogw,mag)
slip = get_slip(length,width,mag)
print(length,width,slip)

In [28]:
subfault_params = banda.subfault_split(lat,lon,length,width,slip,depth_offset=deltadepth)

In [29]:
fault = tb.maketopo.make_fault_dtopo(subfault_params,bounds)

In [35]:
fault.plot_subfaults()
cs = plt.contour(X,Y,banda.depth_map(Y,X).T,levels=[5000,10000,25000,50000])
plt.clabel(cs)
plt.show()

In [37]:
d = banda.depth.copy()
d[d==0] = np.nan

In [43]:
Lat,Lon = np.meshgrid(banda.lat,banda.lon)
plt.pcolormesh(Lon,Lat,d.T)
plt.scatter([132.225436],[-3.881648])

In [34]:
banda.depth_map(-3.881648,132.225436)

In [39]:
subfault_params.loc[4]

In [44]:
def split_rect(fault,lat,lon,length,width,deltadepth,n=11,m=3):
    R = fault.R
    # n = int(length/15000)
    # m = int(width/15000)
    n_steps = 8
    length_step = length/(n*n_steps)
    width_step = width/(m*n_steps)
    sublength = length/n
    subwidth = width/m

    lats = np.empty(n)
    lons = np.empty(n)
    lats[(n - 1)//2] = lat
    lons[(n - 1)//2] = lon

    # add strikeward and anti-strikeward centers
    bearing1 = fault.strike_from_lat_lon(lat,lon)
    bearing2 = (bearing1-180)%360
    lat1,lon1 = lat,lon
    lat2,lon2 = lat,lon
    for i in range(1,(n - 1)//2+1):
        for j in range(n_steps):
            lat1,lon1 = Fault.displace(lat1,lon1,bearing1,length_step,R)
            lat2,lon2 = Fault.displace(lat2,lon2,bearing2,length_step,R)
            bearing1 = fault.strike_from_lat_lon(lat1, lon1)
            bearing2 = (fault.strike_from_lat_lon(lat2, lon2)-180)%360
        lats[(n-1)//2+i] = lat1
        lats[(n-1)//2-i] = lat2
        lons[(n-1)//2+i] = lon1
        lons[(n-1)//2-i] = lon2

    strikes = fault.strike_map(np.vstack((lats,lons)).T)
    dips = fault.dip_map(np.vstack((lats,lons)).T)
    dipward = (strikes+90)%360

    Lats = np.empty((m,n))
    Lons = np.empty((m,n))
    Strikes = np.empty((m,n))
    Dips = np.empty((m,n))
    Lats[(m-1)//2] = lats
    Lons[(m-1)//2] = lons
    Strikes[(m-1)//2] = strikes
    Dips[(m-1)//2] = dips

    # add dipward and antidipward centers
    templats1,templons1 = lats.copy(),lons.copy()
    templats2,templons2 = lats.copy(),lons.copy()
    tempdips1,tempdips2 = dips.copy(),dips.copy()
    for i in range(1,(m - 1)//2+1):
        for j in range(n_steps):
            templats1,templons1 = Fault.displace(templats1,templons1,dipward,width_step*np.cos(np.deg2rad(tempdips1)),R)
            templats2,templons2 = Fault.displace(templats2,templons2,dipward,-width_step*np.cos(np.deg2rad(tempdips2)),R)
            tempdips1 = fault.dip_map(np.vstack((templats1,templons1)).T)
            tempdips2 = fault.dip_map(np.vstack((templats2,templons2)).T)
        Lats[(m-1)//2+i] = templats1
        Lats[(m-1)//2-i] = templats2
        Lons[(m-1)//2+i] = templons1
        Lons[(m-1)//2-i] = templons2
        Strikes[(m-1)//2+i] = fault.strike_map(np.vstack((templats1,templons1)).T)
        Strikes[(m-1)//2-i] = fault.strike_map(np.vstack((templats2,templons2)).T)
        Dips[(m-1)//2+i] = tempdips1
        Dips[(m-1)//2-i] = tempdips2

    Depths = fault.depth_map(np.vstack((Lats.flatten(),Lons.flatten())).T) + deltadepth
    data = [Lats,Lons,Strikes,Dips,Depths]
    data = [arr.flatten() for arr in data]
    return np.array(data).T, sublength, subwidth